In [1]:
###  模块与数据导入

import os 
import pandas as pd
import numpy as np
import scipy
import missingno as msno
import lightgbm as lgb
import xgboost as xgb
import featuretools as ft
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler,StandardScaler,Normalizer,PolynomialFeatures

Train_data = pd.read_csv("Dataset/train/outputs/主蒸汽流量.csv")

for path, dirs, files in os.walk("Dataset/train/inputs"):
    for file in files:
        temp = pd.read_csv(os.path.join(path, file))
        temp.drop_duplicates('时间', inplace=True)
        Train_data = Train_data.merge(temp, on=['时间'], how='left')

Test_data = pd.read_csv("Dataset/test/二次风调门.csv")['时间'].to_frame()
for path, dirs, files in os.walk("Dataset/test"):
    for file in files:
        temp = pd.read_csv(os.path.join(path, file))
        temp.drop_duplicates('时间', inplace=True)
        Test_data = Test_data.merge(temp, on=['时间'], how='left')

Train_data['Is_Test'] = 0
Test_data['Is_Test'] = 1

combi = pd.concat([Train_data, Test_data], axis=0, ignore_index=True)

combi.rename(columns={'时间':'Time', '主蒸汽流量':'Steam_flow', '二次风调门':'2_air_door', '炉排启停':'Grate_switch', '二次风量':'2wind',\
     '一次风量':'1wind', '一次风调门':'1_air_door', '氧量设定值':'O2', '推料器自动投退信号':'Push_auto_feed', 'SO2含量':'SO2',
      '推料器手动指令':'Manual_feed', 'CO含量':'CO', '主蒸汽流量设定值':'Steam_flow_set', '汽包水位':'Water_level', '推料器自动指令':'Push_auto',
       'HCL含量':'HCL','NOx含量':'NOx', '炉排实际运行指令':'Grate_run', '炉排手动指令':'Grate_manual', '给水流量':'Water_flow',
       '炉排自动投退信号':'Grate_auto', '推料器启停':'Feed_switch', '引风机转速':'Fan_speed'}, inplace=True)

In [2]:
# 时间信息拆解
combi['days'] = pd.to_datetime(combi['Time'],format='%Y-%m-%d %H:%M:%S').dt.day
combi['hours'] = pd.to_datetime(combi['Time'],format='%Y-%m-%d %H:%M:%S').dt.hour
combi['minutes'] = pd.to_datetime(combi['Time'],format='%Y-%m-%d %H:%M:%S').dt.minute
combi['seconds'] = pd.to_datetime(combi['Time'],format='%Y-%m-%d %H:%M:%S').dt.second

In [3]:
### 通用函数定义

def feat_kernelMedian(df, df_feature, fe, value, pr, name=""):
    def get_median(a, pr=pr):
        a = np.array(a)
        x = a[~np.isnan(a)]
        n = len(x)
        weight = np.repeat(1.0, n)
        idx = np.argsort(x)
        x = x[idx]
        if n<pr.shape[0]:
            pr = pr[n,:n]
        else:
            scale = (n-1)/2.
            xxx = np.arange(-(n+1)/2.+1, (n+1)/2., step=1)/scale
            yyy = 3./4.*(1-xxx**2)
            yyy = yyy/np.sum(yyy)
            pr = (yyy*n+1)/(n+1)
        ans = np.sum(pr*x*weight) / float(np.sum(pr * weight))
        return ans

    df_count = pd.DataFrame(df_feature.groupby(fe)[value].apply(get_median)).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_mean" % ("_".join(fe))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left").fillna(0)
    return df
    
def merge_mean(df,columns,value,cname):
    add = pd.DataFrame(df.groupby(columns)[value].mean()).reset_index()
    add.columns=columns+[cname]
    df=df.merge(add,on=columns,how="left")
    return df

def outliers_proc(data, col_name, scale=3):
    """
    用于清洗异常值，默认用 box_plot（scale=3）进行清洗
    :param data: 接收 pandas 数据格式
    :param col_name: pandas 列名
    :param scale: 尺度
    :return:
    """

    def box_plot_outliers(data_ser, box_scale):
        """
        利用箱线图去除异常值
        :param data_ser: 接收 pandas.Series 数据格式
        :param box_scale: 箱线图尺度，
        :return:
        """
        iqr = box_scale * (data_ser.quantile(0.75) - data_ser.quantile(0.25))
        val_low = data_ser.quantile(0.25) - iqr
        val_up = data_ser.quantile(0.75) + iqr
        rule_low = (data_ser < val_low)
        rule_up = (data_ser > val_up)
        return (rule_low, rule_up), (val_low, val_up)

    data_n = data.copy()
    data_series = data_n[col_name]
    rule, value = box_plot_outliers(data_series, box_scale=scale)
    index = np.arange(data_series.shape[0])[rule[0] | rule[1]]
    print("Delete number is: {}".format(len(index)))
    data_n = data_n.drop(index)
    data_n.reset_index(drop=True, inplace=True)
    print("Now column number is: {}".format(data_n.shape[0]))
    index_low = np.arange(data_series.shape[0])[rule[0]]
    outliers = data_series.iloc[index_low]
    print("Description of data less than the lower bound is:")
    print(pd.Series(outliers).describe())
    index_up = np.arange(data_series.shape[0])[rule[1]]
    outliers = data_series.iloc[index_up]
    print("Description of data larger than the upper bound is:")
    print(pd.Series(outliers).describe())
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 7))
    sns.boxplot(y=data[col_name], data=data, palette="Set1", ax=ax[0])
    sns.boxplot(y=data_n[col_name], data=data_n, palette="Set1", ax=ax[1])
    return data_n

# Feiyang: 1. 获得核函数 PrEp
PrOriginalEp = np.zeros((2000,2000))
PrOriginalEp[1,0] = 1
PrOriginalEp[2,range(2)] = [0.5,0.5]
for i in range(3,2000):
    scale = (i-1)/2.
    x = np.arange(-(i+1)/2.+1, (i+1)/2., step=1)/scale
    y = 3./4.*(1-x**2)
    y = y/np.sum(y)
    PrOriginalEp[i, range(i)] = y
PrEp = PrOriginalEp.copy()
for i in range(3, 2000):
    PrEp[i,:i] = (PrEp[i,:i]*i+1)/(i+1)

In [4]:
### 参数设置
Missing_value_proc = 2
outliers_proc = False
poly_features = False
linear_features = False
scaler_proc = False
boxcox_proc = False

In [5]:
# 异常值处理 # TODO
if outliers_proc:
    combi = outliers_proc(combi, 'Radiation', scale=3)
    combi = outliers_proc(combi, 'Temp', scale=3)
    combi = outliers_proc(combi, 'Spd', scale=3)
    combi = outliers_proc(combi, 'Dir', scale=3)
else:
    pass

In [6]:
### 缺失值处理
# 对lightgbm和xgboost来说，缺失值处理不是必须的，但是对于其他模型来说，缺失值处理是必须的
# 选择1，用前一个值填充
# 选择2，丢弃缺失值
# 选择3，模型预测填充
# 选择4, 不处理
if Missing_value_proc == 1:
    combi['Temp'] = combi['Temp'].fillna(method='ffill')
    combi['Spd'] = combi['Spd'].fillna(method='ffill')
    combi['Dir'] = combi['Dir'].fillna(method='ffill')
elif Missing_value_proc == 2:
    combi.dropna(axis=0,how='any',subset=['Grate_switch'], inplace=True)
elif Missing_value_proc == 3:
    pass
elif Missing_value_proc == 4:
    pass

In [7]:
### 生成一些新的信息
combi['Water_flow_shift'] = combi['Water_flow'].shift(-276)
combi['Water_flow_mean']  = combi['Water_flow_shift'].rolling(window=552, min_periods=1, center=True).mean()
combi['Water_level_diff'] = combi['Water_level'].diff(20)

Water_level_cum = (np.cumsum(combi['Water_level']-combi['Water_level'].mean())/200).shift(-61)
combi['Water_level_cum'] = Water_level_cum + combi['Water_flow']
combi['Water_level_cum'][-61:] = combi['Water_level_cum'][258992:259053]+14.3352
combi['Water_level_cum_rolling'] = combi['Water_level_cum'].rolling(3000, center=True, min_periods=1).mean()
combi['Steam_flow_rolling'] = combi['Steam_flow'].rolling(3000, center=True, min_periods=1).mean()
combi['simu'] = combi['Water_level_cum'] - combi['Water_level_cum_rolling'] + combi['Steam_flow_rolling']
combi['simu'][-1800:] = combi['Water_level_cum'][-1800:]

In [8]:
### 特征组合

# 多项式特征
if poly_features:
    combi_poly = combi[['Temp', 'Spd', 'Temp_day_mean', 'Temp_day_median', 'Temp_hour_mean', 'Temp_hour_median', \
                        'Ratio_Hour_median', 'Radiation_hour_mean', 'Radiation_hour_median']]
    poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
    combi_poly = poly.fit_transform(combi_poly)
    combi_poly = pd.DataFrame(combi_poly, columns=poly.get_feature_names_out())

# 线性特征
elif linear_features:
    es = ft.EntitySet(id="radia")
    es.add_dataframe(dataframe_name="Elec3", dataframe=combi, index="index")
    
    trans_primitives=['add_numeric', 'subtract_numeric', 'multiply_numeric', 'divide_numeric']
    agg_primitives=['sum', 'median','mean']
    combi_linear, feature_names = ft.dfs(entityset=es, 
                                           target_dataframe_name = 'Elec3', 
                                           max_depth = 1, 
                                           verbose = 1,
                                           agg_primitives=agg_primitives,
                                           trans_primitives=trans_primitives,
                                           n_jobs = 8)
    
    combi_linear = combi_linear.reindex(index=combi["index"])
    combi_linear = combi_linear.reset_index()

In [9]:
# 数据归一化与标准化
features = combi.select_dtypes(include=['int64', 'float64']).drop('Steam_flow', axis=1).columns
if scaler_proc == True:
    scaler = [PolynomialFeatures(len(features)), MinMaxScaler(), StandardScaler(), Normalizer()]
    x_train=scaler[1].fit_transform(combi[features])

    for i in np.arange(len(features)):
        combi[features[i]] = x_train[:, i]

In [10]:
# boxcox
features = combi.select_dtypes(include=['int64', 'float64']).drop('Steam_flow', axis=1).columns
if boxcox_proc == True:
    for i in np.arange(len(features)):
        combi[features[i]] = scipy.special.boxcox1p(combi[features[i]], 0)
        # combi_copy[features[i]], _ = stats.boxcox(combi[features[i]]+0.01)

In [11]:
# # for deep learning
# dcd = pd.read_csv("Dataset/decomposed_data.csv")
# combi['date'] = dcd['ds']
# combi = combi[combi['Day'] < 300]
# combi = combi[['date','Temp', 'Spd', 'Temp_day_median', 'Spd_day_median', 'Diff_Temp_Day', 'Radiation_hour_median',\
#             'Diff_Temp_Hour','Temp_day_max_min', 'Temp_day_max_median', 'Temp_day_relative', 'Radiation']]
# combi.to_csv("/mnt/DATA1/wuliubin/Download/Elec3/LTSF-Linear/dataset/elec3.csv", index=False, sep=',')

In [12]:
combi.to_csv("Dataset/combi.csv", index=False, sep=',')